## Evaluation Baseline

In [1]:
## Import
from sklearn.cluster import KMeans
import collections, numpy
from scipy import spatial
import numpy as np
import librosa
import math
import re
import os, sys
import pandas as pd
import IPython.display as ipd
import pickle

In [2]:
# Load metadata and features.
tracks = pd.read_csv('fma_metadata/tracks.csv')
r_tracks = pd.read_csv('fma_metadata/raw_tracks.csv')
genres = pd.read_csv('fma_metadata/genres.csv')
features = pd.read_csv('fma_metadata/features.csv')
echonest = pd.read_csv('fma_metadata/echonest.csv')
r_artists = pd.read_csv('fma_metadata/raw_artists.csv')

/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170

In [3]:
# overall subset
tracks_sm = tracks.loc[(tracks['set.1'] == 'small') & (tracks['set'] == 'training')]

In [4]:
# subset based on artists in set
# load
def load_spot_rel_artists(name):
    with open(name, 'rb') as handle:
        thing = pickle.load(handle)   
    return thing
artist_rel = load_spot_rel_artists("saved_data/artist_rel_small_training.pickle")


# ipd.display(features.columns[253:400])


In [5]:
len(artist_rel)

1026

In [6]:
# Subsetting Business, Creating Master tables

# Creating Condensed Artist Map, and Master 2D Table
artist_all = {}

for key in artist_rel:
    artist_all[key] = set()

for key in artist_rel:
    for elem in artist_rel[key]:
        if artist_rel.get(elem, None) is not None:
            artist_all[key].add(elem)
            artist_all[elem].add(key)

# assert len(artist_all) == len(artist_rel)  
# deleting artists with no related artist in set
keys = artist_all.keys()
for key in keys:
    if not artist_all[key]:
        del artist_all[key]
#         pass
    else:
        artist_all[key] = list(artist_all[key])
        
# Artist Index Map
artist_index = {}
for i, a in enumerate(artist_all.keys()):
    artist_index[a] = i


In [7]:
artist_tracks = tracks_sm.loc[(tracks_sm['artist.12'] == artist_all.keys()[4])]

In [8]:
artist_tracks['Unnamed: 0']

19350    30690
22365    34996
25121    38847
29414    44342
31871    48307
Name: Unnamed: 0, dtype: object

In [13]:
class FeatureData:

    'Music audio features for genre classification'
    hop_length = None

    dir_trainfolder = "/Users/anirudhmani/Prog/thesis_related/data/fma_small_consolidated"
    dir_devfolder = "/Users/anirudhmani/Prog/thesis_related/data/fma_small/001"
    dir_testfolder = "/Users/anirudhmani/Prog/thesis_related/data/fma_small/002"
#     dir_all_files = "/Users/anirudhmani/Prog/thesis_related/data"

    train_X_preprocessed_data = 'data_train_input.npy'
    train_Y_preprocessed_data = 'data_train_target.npy'
    dev_X_preprocessed_data = 'data_validation_input.npy'
    dev_Y_preprocessed_data = 'data_validation_target.npy'
    test_X_preprocessed_data = 'data_test_input.npy'
    test_Y_preprocessed_data = 'data_test_target.npy'

    train_X = train_Y = None
    dev_X = dev_Y = None
    test_X = test_Y = None

    def __init__(self):
        self.hop_length = 512
        self.timeseries_length_list = []

    def load_preprocess_data(self): 
        self.trainfiles_list = self.path_to_audiofiles(self.dir_trainfolder)
        self.devfiles_list = self.path_to_audiofiles(self.dir_devfolder)
        self.testfiles_list = self.path_to_audiofiles(self.dir_testfolder)

        all_files_list = []
        all_files_list.extend(self.trainfiles_list)
        all_files_list.extend(self.devfiles_list)
        all_files_list.extend(self.testfiles_list)

        #self.precompute_min_timeseries_len(all_files_list)
        print("[DEBUG] total number of files: " + str(len(self.timeseries_length_list)))

        # Training set
        self.train_X = self.extract_audio_features(self.trainfiles_list)
        with open(self.train_X_preprocessed_data, 'wb') as f:
            np.save(f, self.train_X)
#         with open(self.train_Y_preprocessed_data, 'wb') as f:
#             self.train_Y = self.one_hot(self.train_Y)
#             np.save(f, self.train_Y)

        # Validation set
#         self.dev_X, self.dev_Y = self.extract_audio_features(self.devfiles_list)
#         with open(self.dev_X_preprocessed_data, 'wb') as f:
#             np.save(f, self.dev_X)
#         with open(self.dev_Y_preprocessed_data, 'wb') as f:
#             self.dev_Y = self.one_hot(self.dev_Y)
#             np.save(f, self.dev_Y)

        # Test set
#         self.test_X, self.test_Y = self.extract_audio_features(self.testfiles_list)
#         with open(self.test_X_preprocessed_data, 'wb') as f:
#             np.save(f, self.test_X)
#         with open(self.test_Y_preprocessed_data, 'wb') as f:
#             self.test_Y = self.one_hot(self.test_Y)
#             np.save(f, self.test_Y)

    def load_deserialize_data(self):

        self.train_X = np.load(self.train_X_preprocessed_data)
#         self.train_Y = np.load(self.train_Y_preprocessed_data)

        self.dev_X = np.load(self.dev_X_preprocessed_data)
#         self.dev_Y = np.load(self.dev_Y_preprocessed_data)

        self.test_X = np.load(self.test_X_preprocessed_data)
#         self.test_Y = np.load(self.test_Y_preprocessed_data)

    def precompute_min_timeseries_len(self, list_of_audiofiles):
        for file in list_of_audiofiles:
            print("Loading " + str(file))
            y, sr = librosa.load(file)
            self.timeseries_length_list.append(math.ceil(len(y) / self.hop_length))

    def extract_audio_features(self, list_of_audiofiles):
        #timeseries_length = min(self.timeseries_length_list)
        timeseries_length = 128 # was 128 previously
        t_length = 12
        data = np.zeros((len(list_of_audiofiles), timeseries_length, 13), dtype=np.float64)
        target = []

        for i, file in enumerate(list_of_audiofiles):
            try:
                y, sr = librosa.load(file)
                       
                mfcc = []
                spectral_center = []
                chroma = []
                spectral_contrast = []

        
                mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=self.hop_length, n_mfcc=13)
#             spectral_center = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=self.hop_length)
#             chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=self.hop_length)
#             spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=self.hop_length)
            
            

                splits = re.split('[ .]', file)
#             genre = re.split('[ /]', splits[1])[3]
#             target.append(genre)


            
                data[i, :, 0:13] = mfcc.T[0:timeseries_length, :]
#             data[i, :, 13:14] = spectral_center.T[0:timeseries_length, :]
#             data[i, :, 14:26] = chroma.T[0:timeseries_length, :]
#             data[i, :, 26:33] = spectral_contrast.T[0:timeseries_length, :]

#             print("Extracted features audio track %i of %i." % (i + 1, len(list_of_audiofiles)))

            except:
                print "error reading.....continuing"
                continue

        print("Extracted audio features .....")
        return data

    def one_hot(self, Y_genre_strings):
        y_one_hot = np.zeros((Y_genre_strings.shape[0], len(self.genre_list)))
        for i, genre_string in enumerate(Y_genre_strings):
            index = self.genre_list.index(genre_string)
            y_one_hot[i, index] = 1
        return y_one_hot

    def path_to_audiofiles(self, dir_folder):
        list_of_audio = []
        for file in os.listdir(dir_folder):
            if file.endswith(".mp3"):
                directory = "%s/%s" % (dir_folder, file)
                list_of_audio.append(directory)
        return list_of_audio
    
    def get_meaned(self, tracks_data, ax):     
        return np.mean(tracks_data, axis=ax)
        


In [14]:
song_features = FeatureData()

In [15]:
# trainfiles_list = song_features.path_to_audiofiles(song_features.dir_trainfolder)

In [ ]:
song_features.load_preprocess_data()

In [25]:
# creating a table ntry for pull path of track

# artist_tracks['track_path'] = "%s/%s" % (song_features.dir_trainfolder, artist_tracks['Unnamed: 0'])

/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
# trainfiles_list[0].split("/")[-1].split(".")[0]

# changing trainfiles list per artist
artist_tracklist = {}
for i, a in enumerate(artist_all.keys()):
    for s in tracks_sm.loc[(tracks_sm['artist.12'] == a)]['Unnamed: 0']:
        
        if a in artist_tracklist:
            artist_tracklist[a].append("%s/%s.mp3" % (song_features.dir_trainfolder, s))  
        else:
            artist_tracklist[a] = ["%s/%s.mp3" % (song_features.dir_trainfolder, s)]


In [82]:
tracks_data = song_features.train_X # num of tracks * Time Length * Num of Features
tracks_data = tracks_data.reshape(tracks_data.shape[0]*tracks_data.shape[1], tracks_data.shape[2])

In [65]:
# tracks_data = song_features.get_meaned(tracks_data, 1) # get mean of features per track, num of tracks * Num of Features

In [84]:
def get_bog_model(data, num_clusters):
    # all features in all songs in all artists
    # X = get_data
#     A = 100
#     F = 13
#     all_features = numpy.random.rand(A, F)
    model = KMeans(num_clusters)
    model.fit(data)
    return model

In [79]:
model = get_bog_model(tracks_data, 100)

In [83]:
def save_spot_rel_artists(name, table):
    with open('saved_data/%s.pickle'%name, 'wb') as writer:
        pickle.dump(table, writer, protocol=pickle.HIGHEST_PROTOCOL)

(7936, 13)

In [ ]:
save_spot_rel_artists('cluster_model', model)

In [37]:
# Output is num_artists * num_clusters 
def construct_artist_similarity_matrix(model, data_folder, num_clusters, num_artists):
    artist_similarity_matrix = numpy.zeros((num_artists, num_clusters))

    # for artist_index, artist in enumerate(data):
    for artist_index, artist in enumerate(range(num_artists)):
        artist_feature = []
        # for song in artist:
        for song in range(10):
            song_features = []
            # for clip in song:
            for clip in range(10):
                # is of 1 X F dimensions
                # features = get_feature(clip)
                features = numpy.random.rand(1,13)
                song_features.append(features)
                
            song_features = numpy.vstack(song_features)
            artist_feature.append(get_histogram_song(song_features, model, num_clusters))
        artist_feature = numpy.vstack(artist_feature)
        artist_similarity_matrix[artist_index, :] = get_artist_feature(artist_feature)
    return artist_similarity_matrix

In [ ]:
artist_similarity_matrix = construct_artist_similarity_matrix(model, data_folder, num_clusters, num_artists)